# Cluster countries

In [ ]:
!pip install umap

  Preparing metadata (setup.py) ... done
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3543 sha256=0206c61fdf3d8d9ff22ecc63f53b8423ebefdba92f00c0fffe4d5fb60f2752ea
  Stored in directory: /root/.cache/pip/wheels/15/f1/28/53dcf7a309118ed35d810a5f9cb995217800f3f269ab5771cb
Successfully built umap


In [ ]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install pandas matplotlib datashader bokeh holoviews scikit-image colorcet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 67.5 MB/s eta 0:00:00


Import the required modules.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.base import TransformerMixin
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import umap
import umap.umap_ as umap_
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, PanTool, ResetTool, BoxZoomTool
from bokeh.palettes import brewer, RdBu, Viridis
output_notebook()

In [ ]:
from google.colab import files

# Upload the dataset file from your local drive to Google Colab
uploaded = files.upload()

# Once the file is uploaded, you can access it like this
for filename in uploaded.keys():
    print('Uploaded file "{name}" with length {length} bytes'.format(
        name=filename, length=len(uploaded[filename])))

Saving world_happiness_2015.csv to world_happiness_2015.csv
Uploaded file "world_happiness_2015.csv" with length 16557 bytes


In [ ]:
import pandas as pd

# Assuming the uploaded file is a CSV file, you can read it into a pandas DataFrame
for filename in uploaded.keys():
    data_2015 = pd.read_csv(filename)
    break  # Assuming there's only one file uploaded, so we break after reading the first file

## Problem setting

Let's try to cluster the countries baased on the indicators of happiness.

## Import data

In [ ]:
# data_2015 = pd.read_csv('Data/world_happiness_2015.csv')

In [ ]:
data_2015.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [ ]:
data_2015.describe()

,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
count,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000
mean,79.493671,5.375734,0.047885,0.846137,0.991046,0.630259,0.428615,0.143422,0.237296,2.098977
std,45.754363,1.145010,0.017146,0.403121,0.272369,0.247078,0.150693,0.120034,0.126685,0.553550
min,1.000000,2.839000,0.018480,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.328580
25%,40.250000,4.526000,0.037268,0.545808,0.856823,0.439185,0.328330,0.061675,0.150553,1.759410
50%,79.500000,5.232500,0.043940,0.910245,1.029510,0.696705,0.435515,0.107220,0.216130,2.095415
75%,118.750000,6.243750,0.052300,1.158448,1.214405,0.811013,0.549092,0.180255,0.309883,2.462415
max,158.000000,7.587000,0.136930,1.690420,1.402230,1.025250,0.669730,0.551910,0.795880,3.602140


## Preprocess data

Define a class to extract data from a pandas DataFrame.

In [ ]:
class FeatureSelector(BaseException, TransformerMixin):
    def __init__(self, feature_names):
        self._feature_names = feature_names
    def fit(self, X, Y=None):
        return self
    def transform(self, X):
        return X[self._feature_names].values

Define a transformer for the region names into one-hot encoding.

In [ ]:
region_transformer = ColumnTransformer([('one_hot_encoder',
                                         OneHotEncoder(categories='auto'),
                                         ['Region'])])

Define the names of the columns that hold numerical data.

In [ ]:
num_attr_names = ['Economy (GDP per Capita)',
                  'Family', 'Health (Life Expectancy)', 'Freedom',
                  'Trust (Government Corruption)', 'Generosity',
                  'Dystopia Residual']

Create a pipeline for the numerical attributes, rescaling them after selection.

In [ ]:
num_attrs_pipeline = Pipeline([
    ('select_num_attrs', FeatureSelector(num_attr_names)),
    ('scaler', MinMaxScaler()),
])

In [ ]:
preparation_pipeline = FeatureUnion(transformer_list=[
    ('region_attr', region_transformer),
    ('num_attrs', num_attrs_pipeline),
])

Run the pipeline to prepare the data.

In [ ]:
prepared_data = num_attrs_pipeline.fit_transform(data_2015)

In [ ]:
prepared_data.shape

(158, 7)

## Dimensionality reduction

### Principal Component Analysis (PCA)

Perform a principal Component Analysis on the data.

In [ ]:
pca = PCA()
pca_data = pd.DataFrame(pca.fit_transform(prepared_data))
pca_data.columns = [f'PC{i}' for i in range(1, 8)]

We can check the variance ratio explained by each of the components.

In [ ]:
pca.explained_variance_ratio_

array([0.47835186, 0.18587618, 0.10516347, 0.08630442, 0.0645048 ,
       0.0532306 , 0.02656866])

It is clear that most of the variance in the data is explained by the first two components, since this amounts to 65 %.  We can visualize the data in two dimension using the two principal components that explain most of the varianace.

In order to assess the results, we want a plot that shows the country name, and the happiness score as well.  The country name can be added using a hover tool, the happiness score as the color of the glyphs.

We add the country name and the happiness score to the PCA dataframe, as well as an additional column that encodes the score as a color from a red-blue Brewer color scheme.  The happiness score is converted to categorical data for this purpose.

In [ ]:
pca_data['Country'] = data_2015['Country']
pca_data['Happiness Score'] = data_2015['Happiness Score']
happiness_min = data_2015['Happiness Score'].min()
happiness_max = data_2015['Happiness Score'].max()
nr_colors = int(happiness_max) - int(happiness_min) + 1
pca_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                     bins=np.linspace(np.floor(happiness_min),
                                                      np.ceil(happiness_max),
                                                      nr_colors + 1),
                                     labels=brewer['RdBu'][nr_colors])

In [ ]:
pca_data

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,Country,Happiness Score,Happiness Color
0,-0.742560,0.255859,-0.046819,-0.200890,-0.005302,-0.030317,-0.030825,Switzerland,7.587,#b2182b
1,-0.573766,0.001956,-0.225616,0.182013,-0.140456,-0.083132,0.025860,Iceland,7.561,#b2182b
2,-0.717045,0.366585,-0.018777,-0.253232,0.000190,-0.112461,-0.032102,Denmark,7.527,#b2182b
3,-0.706379,0.232345,-0.063994,-0.109291,-0.012735,-0.043326,0.057998,Norway,7.522,#b2182b
4,-0.643131,0.236638,-0.055096,-0.007588,-0.114805,-0.131293,0.032526,Canada,7.427,#b2182b
...,...,...,...,...,...,...,...,...,...,...
153,0.065691,0.797183,0.437880,-0.209177,0.248768,0.066833,-0.235110,Rwanda,3.465,#67a9cf
154,0.531902,0.232314,0.097846,0.080434,-0.014925,0.311838,0.053203,Benin,3.340,#67a9cf
155,0.274434,0.029431,0.737073,0.132340,-0.209417,-0.158671,0.001287,Syria,3.006,#67a9cf
156,0.855529,0.047117,0.183211,-0.130877,-0.079312,-0.056171,-0.046203,Burundi,2.905,#2166ac


Now we can create a plot with the PCA dataframe as source for the glyphs and their color, as well as the tooltip information that shows the country name.

In [ ]:
hovertool = HoverTool(tooltips=[('Country', '@Country')])
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)
fig.circle('PC1', 'PC2', source=pca_data, fill_color='Happiness Color',
           size=8, alpha=0.85)
show(fig)

It is clear from the plot above that the first principal component seems to be a good indicator for the countries with a high happiness score (colored red).

### Linear Discriminant Analysis (LDA)

Linear Discriminant Analysis is a supervised learning technique that maximizes the between-category distances, while minimizing the within-category-distance.

We create labels for the data by introducing categories based on the happiness score.

In [ ]:
happiness = pd.DataFrame()
happiness['Country'] = data_2015.Country
happiness['Happiness Label'] = pd.cut(data_2015['Happiness Score'],
                                      bins=[0.0, 4.0, 7.0, 10.0],
                                      labels=['unhappy', 'neutral', 'happy'])

Most countries are in the 'neutral' category, few in 'happy' or 'unhappy'.  Note that the label names are not normative.

In [ ]:
happiness.groupby('Happiness Label').count()

,Country
Happiness Label,
unhappy,21
neutral,122
happy,15


LDA is a supervised method, so we use the happiness label as output.

In [ ]:
target = happiness['Happiness Label'].values

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=2)
lda_data = pd.DataFrame(lda.fit(prepared_data, target) \
                           .transform(prepared_data))
lda_data.columns = ['C1', 'C2']

The LDA produces two components for each country, and we can plot the countries using those similar as we did for the PCA.

In [ ]:
lda_data['Country'] = data_2015['Country']
lda_data['Happiness Score'] = data_2015['Happiness Score']
lda_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                     bins=[0.0, 4.0, 7.0, 10.0],
                                     labels=['blue', 'yellow', 'red'])

In [ ]:
hovertool = HoverTool(tooltips=[('Country', '@Country')])
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)
fig.circle('C1', 'C2', source=lda_data, fill_color='Happiness Color',
           size=8, alpha=0.85)
show(fig)

### t-SNE

In [ ]:
tsne_data = pd.DataFrame(TSNE(learning_rate=70).fit_transform(prepared_data))
tsne_data.columns = ['C1', 'C2']

In [ ]:
tsne_data['Country'] = data_2015['Country']
tsne_data['Happiness Score'] = data_2015['Happiness Score']
tsne_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                      bins=[0.0, 4.0, 7.0, 10.0],
                                      labels=['blue', 'yellow', 'red'])

In [ ]:
tsne_data.head()

,C1,C2,Country,Happiness Score,Happiness Color
0,11.010277,-1.548814,Switzerland,7.587,red
1,9.361794,1.146479,Iceland,7.561,red
2,11.382942,-2.037371,Denmark,7.527,red
3,10.694201,-0.966875,Norway,7.522,red
4,10.803186,-0.192115,Canada,7.427,red


In [ ]:
hovertool = HoverTool(tooltips=[('Country', '@Country')])
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)
fig.circle('C1', 'C2', source=tsne_data, fill_color='Happiness Color',
           size=8, alpha=0.85)
show(fig)

### UMAP

In this section, we add dimensionality reduction using UMAP. We experiment with the following hyperparameters (n_neighbours, min_distance and metric) to see whether and how they influence the resulting visualization.

### Number of neighbours

This parameter controls how UMAP balances local versus global structure in the data. It has a default value of 15. We will explore three iterations on n_neighbours: 2, 15 and 30

In [ ]:
# Iteration 1: n_neighbors=2, min_dist=0.1, n_components=2

# Perform dimensionality reduction using UMAP
umap_data = umap.UMAP(n_neighbors=2, min_dist=0.1, n_components=2).fit_transform(prepared_data)
umap_data = pd.DataFrame(umap_data, columns=['C1', 'C2'])

# Add 'Country', 'Happiness Score', and 'Happiness Color' columns to the UMAP transformed data
umap_data['Country'] = data_2015['Country']
umap_data['Happiness Score'] = data_2015['Happiness Score']
umap_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                      bins=[0.0, 4.0, 7.0, 10.0],
                                      labels=['blue', 'yellow', 'red'])

# Create HoverTool
hovertool = HoverTool(tooltips=[('Country', '@Country'), ('Happiness Score', '@{Happiness Score}')])

# Create Bokeh plot
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)

# Plotting the UMAP transformed data
fig.circle('C1', 'C2', source=umap_data, fill_color='Happiness Color',
           size=9, alpha=0.7)

# Show the plot
show(fig)

In [ ]:
# Iteration 2: n_neighbors=15, min_dist=0.1, n_components=2

# Perform dimensionality reduction using UMAP
umap_data = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2).fit_transform(prepared_data)
umap_data = pd.DataFrame(umap_data, columns=['C1', 'C2'])

# Add 'Country', 'Happiness Score', and 'Happiness Color' columns to the UMAP transformed data
umap_data['Country'] = data_2015['Country']
umap_data['Happiness Score'] = data_2015['Happiness Score']
umap_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                      bins=[0.0, 4.0, 7.0, 10.0],
                                      labels=['blue', 'yellow', 'red'])

# Create HoverTool
hovertool = HoverTool(tooltips=[('Country', '@Country'), ('Happiness Score', '@{Happiness Score}')])

# Create Bokeh plot
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)

# Plotting the UMAP transformed data
fig.circle('C1', 'C2', source=umap_data, fill_color='Happiness Color',
           size=9, alpha=0.7)

# Show the plot
show(fig)

In [ ]:
# Iteration 3: n_neighbors=30, min_dist=0.1, n_components=2

# Perform dimensionality reduction using UMAP
umap_data = umap.UMAP(n_neighbors=30, min_dist=0.1, n_components=2).fit_transform(prepared_data)
umap_data = pd.DataFrame(umap_data, columns=['C1', 'C2'])

# Add 'Country', 'Happiness Score', and 'Happiness Color' columns to the UMAP transformed data
umap_data['Country'] = data_2015['Country']
umap_data['Happiness Score'] = data_2015['Happiness Score']
umap_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                      bins=[0.0, 4.0, 7.0, 10.0],
                                      labels=['blue', 'yellow', 'red'])

# Create HoverTool
hovertool = HoverTool(tooltips=[('Country', '@Country'), ('Happiness Score', '@{Happiness Score}')])

# Create Bokeh plot
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)

# Plotting the UMAP transformed data
fig.circle('C1', 'C2', source=umap_data, fill_color='Happiness Color',
           size=9, alpha=0.7)

# Show the plot
show(fig)

The three iterations above for number of neighbours show clearly different patterns for each of the hyperparameter values. For n_neighbors=2, the points are very disconnected and scattered throughout the space, so no clear clusters of happiness levels can be seen.

As n_neighbors is increased to 15, a more clearer structure emerges, with high happiness cluster clearly separated from low and medium. The last two are intermingled a little bit. Overall, there is a fairly good overall view of the data showing how the various colors interelate to each other.

As n_neighbors increases to 30, much more focus in placed on the overall structure of the data. The overall structure is relatively well captured, although there is room for improvement especially for the red and yellow.

### Minimum distance

The min_dist parameter provides the minimum distance apart that points are allowed to be in the low dimensional representation. The default value for min_dist is 0.1. We will iterate using the following discrete values: 0.0, 0.1 and 0.99.

In [ ]:
# Iteration 4: n_neighbors=15, min_dist=0.0, n_components=2

# Perform dimensionality reduction using UMAP
umap_data = umap.UMAP(n_neighbors=15, min_dist=0.0, n_components=2).fit_transform(prepared_data)
umap_data = pd.DataFrame(umap_data, columns=['C1', 'C2'])

# Add 'Country', 'Happiness Score', and 'Happiness Color' columns to the UMAP transformed data
umap_data['Country'] = data_2015['Country']
umap_data['Happiness Score'] = data_2015['Happiness Score']
umap_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                      bins=[0.0, 4.0, 7.0, 10.0],
                                      labels=['blue', 'yellow', 'red'])

# Create HoverTool
hovertool = HoverTool(tooltips=[('Country', '@Country'), ('Happiness Score', '@{Happiness Score}')])

# Create Bokeh plot
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)

# Plotting the UMAP transformed data
fig.circle('C1', 'C2', source=umap_data, fill_color='Happiness Color',
           size=9, alpha=0.7)

# Show the plot
show(fig)

In [ ]:
# Iteration 5: n_neighbors=15, min_dist=0.1, n_components=2

# Perform dimensionality reduction using UMAP
umap_data = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2).fit_transform(prepared_data)
umap_data = pd.DataFrame(umap_data, columns=['C1', 'C2'])

# Add 'Country', 'Happiness Score', and 'Happiness Color' columns to the UMAP transformed data
umap_data['Country'] = data_2015['Country']
umap_data['Happiness Score'] = data_2015['Happiness Score']
umap_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                      bins=[0.0, 4.0, 7.0, 10.0],
                                      labels=['blue', 'yellow', 'red'])

# Create HoverTool
hovertool = HoverTool(tooltips=[('Country', '@Country'), ('Happiness Score', '@{Happiness Score}')])

# Create Bokeh plot
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)

# Plotting the UMAP transformed data
fig.circle('C1', 'C2', source=umap_data, fill_color='Happiness Color',
           size=9, alpha=0.7)

# Show the plot
show(fig)

In [ ]:
# Iteration 6: n_neighbors=15, min_dist=0.99, n_components=2

# Perform dimensionality reduction using UMAP
umap_data = umap.UMAP(n_neighbors=15, min_dist=0.99, n_components=2).fit_transform(prepared_data)
umap_data = pd.DataFrame(umap_data, columns=['C1', 'C2'])

# Add 'Country', 'Happiness Score', and 'Happiness Color' columns to the UMAP transformed data
umap_data['Country'] = data_2015['Country']
umap_data['Happiness Score'] = data_2015['Happiness Score']
umap_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                      bins=[0.0, 4.0, 7.0, 10.0],
                                      labels=['blue', 'yellow', 'red'])

# Create HoverTool
hovertool = HoverTool(tooltips=[('Country', '@Country'), ('Happiness Score', '@{Happiness Score}')])

# Create Bokeh plot
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)

# Plotting the UMAP transformed data
fig.circle('C1', 'C2', source=umap_data, fill_color='Happiness Color',
           size=9, alpha=0.7)

# Show the plot
show(fig)

The three iterations above for minimum distance show clearly different patterns for each of the hyperparameter values. For min_dist=0.0, UMAP manages to find small connected components, with red and blue clusters at opposite ends and yellow cluster in the middle. As min_dist is increased these structures are pushed apart into softer more general features. The pattern for min_distance=0.99 spans the space more with less gaps between the clusters.

### Metric

This controls how distance is computed in the ambient space of the input data. By default UMAP supports a wide variety of metrics, which can be grouped into Minkowski style metrics, Normalized spatial metrics and Angular and correlation metrics, among others. For the hyperparameter tuning, we selected one metric from each group, namely: Euclidean, mahalanobis and cosine respectively

In [ ]:
# Iteration 7: n_neighbors=15, min_dist=0.1, n_components=2, metric = cosine

# Perform dimensionality reduction using UMAP
umap_data = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric="cosine").fit_transform(prepared_data)
umap_data = pd.DataFrame(umap_data, columns=['C1', 'C2'])

# Add 'Country', 'Happiness Score', and 'Happiness Color' columns to the UMAP transformed data
umap_data['Country'] = data_2015['Country']
umap_data['Happiness Score'] = data_2015['Happiness Score']
umap_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                      bins=[0.0, 4.0, 7.0, 10.0],
                                      labels=['blue', 'yellow', 'red'])

# Create HoverTool
hovertool = HoverTool(tooltips=[('Country', '@Country'), ('Happiness Score', '@{Happiness Score}')])

# Create Bokeh plot
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)

# Plotting the UMAP transformed data
fig.circle('C1', 'C2', source=umap_data, fill_color='Happiness Color',
           size=9, alpha=0.7)

# Show the plot
show(fig)

In [ ]:
# Iteration 8: n_neighbors=15, min_dist=0.1, n_components=2, metric = mahalanobis

# Perform dimensionality reduction using UMAP
umap_data = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric="mahalanobis").fit_transform(prepared_data)
umap_data = pd.DataFrame(umap_data, columns=['C1', 'C2'])

# Add 'Country', 'Happiness Score', and 'Happiness Color' columns to the UMAP transformed data
umap_data['Country'] = data_2015['Country']
umap_data['Happiness Score'] = data_2015['Happiness Score']
umap_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                      bins=[0.0, 4.0, 7.0, 10.0],
                                      labels=['blue', 'yellow', 'red'])

# Create HoverTool
hovertool = HoverTool(tooltips=[('Country', '@Country'), ('Happiness Score', '@{Happiness Score}')])

# Create Bokeh plot
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)

# Plotting the UMAP transformed data
fig.circle('C1', 'C2', source=umap_data, fill_color='Happiness Color',
           size=9, alpha=0.7)

# Show the plot
show(fig)

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:576: RuntimeWarning: overflow encountered in cast
  knn_dists = knn_dists.astype(np.float32)


In [ ]:
# Iteration 9: n_neighbors=15, min_dist=0.1, n_components=2, metric = cosine

# Perform dimensionality reduction using UMAP
umap_data = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric="cosine").fit_transform(prepared_data)
umap_data = pd.DataFrame(umap_data, columns=['C1', 'C2'])

# Add 'Country', 'Happiness Score', and 'Happiness Color' columns to the UMAP transformed data
umap_data['Country'] = data_2015['Country']
umap_data['Happiness Score'] = data_2015['Happiness Score']
umap_data['Happiness Color'] = pd.cut(data_2015['Happiness Score'],
                                      bins=[0.0, 4.0, 7.0, 10.0],
                                      labels=['blue', 'yellow', 'red'])

# Create HoverTool
hovertool = HoverTool(tooltips=[('Country', '@Country'), ('Happiness Score', '@{Happiness Score}')])

# Create Bokeh plot
fig = figure(tools=[PanTool(), BoxZoomTool(), hovertool, ResetTool()],
             width=500, height=400)

# Plotting the UMAP transformed data
fig.circle('C1', 'C2', source=umap_data, fill_color='Happiness Color',
           size=9, alpha=0.7)

# Show the plot
show(fig)

The above plots show the 2 dimensional representations of the happiness scores by countries. While the plots for Euclidean and cosine show some form of clustering, there seems to be no clear clustering. Of the three, mahalanobis shows three clearer clusters of blue and red at opposite ends and yellow in the middle. Thus, the choice of metric has a clear impact on the low dimensional representation.